In [ ]:
import cv2
import os, sys
import shutil

In [ ]:
import cv2
import os
import shutil

ALLOWED_IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png']

class VideoToImageWriter:

    def __init__(
            self, 
            video_path, 
            output_image_path, 
            prefix,
            extension=".jpg", 
            image_name_padding_length=4, 
            num_image_per_directory=100
            ):
        
        self.video_path = video_path
        self.output_image_path = output_image_path
        self.prefix = prefix
        self.extension = extension
        self.image_name_padding_length = image_name_padding_length
        self.num_image_per_directory = num_image_per_directory

    def write_images(self):
        try:
            # Check if the image extension is allowed
            
            if self.extension not in ALLOWED_IMAGE_EXTENSIONS:
                raise ValueError('Invalid image extension provided')

            # Remove the existing output directory and its contents if it exists
            # and make a new one with the same name
            if os.path.exists(self.output_image_path):
                try:
                    shutil.rmtree(self.output_image_path)
                    print("Directory already present")
                    print(f'====> Removing existing directory: {self.output_image_path}...')
                    print()
            
                except OSError as e:
                    print("Error: %s - %s." % (e.filename, e.strerror))

            try:
                print(f'====> Making new directory: {self.output_image_path}...')
                print()
                os.makedirs(self.output_image_path)

            except OSError as e:
                print("Error: %s - %s." % (e.filename, e.strerror))

            cap = cv2.VideoCapture(self.video_path)

            count = 0
            dir_count = 1
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                if count % self.num_image_per_directory == 0:
                    try:
                        print()
                        print(f"=====> Making internal directory: {dir_count}")
                        dir_name = os.path.join(self.output_image_path, str(dir_count))
                        os.makedirs(dir_name, exist_ok=True)

                    except OSError as e:
                        print("Error: %s - %s." % (e.filename, e.strerror))

                    
                    
                    # Create a new directory for the next set of images
                    dir_count += 1



                num = str(count).zfill(self.image_name_padding_length)
                filename = f'{self.prefix}_{num}{self.extension}'
                image_path = os.path.join(dir_name, filename)

                print("Writing file:", image_path)
                cv2.imwrite(image_path, frame)
                count += 1
            cap.release()

        except Exception as e:
            print("An error occurred:", str(e))
        finally:
            if cap is not None:
                cap.release()
                print("Released capture object")

In [ ]:
# Example usage
if __name__ == "__main__":
    video_path = "input-video.mp4"
    output_path = "output_images"
    prefix = "image"
    padding_length = 4
    num_per_directory = 100

    image_writer = VideoToImageWriter(
                        video_path, 
                        output_path, 
                        prefix,
                        image_name_padding_length=padding_length, 
                        num_image_per_directory=num_per_directory
                        )
    image_writer.write_images()
